<img src="https://hadoop.apache.org/hadoop-logo.jpg" align ="left">
<h1><div class="alert alert-block alert-info">
Yet Another Resource Negotiator (YARN)
</div></h1>
<img src="https://i.ytimg.com/vi/ZFbkNY6Xn94/maxresdefault.jpg">

<img src="https://hadoop.apache.org/hadoop-logo.jpg" align ="left">
<h1><div class="alert alert-block alert-info">
Hadoop MapReduce
</div></h1>

Hadoop MapReduce is a software framework for easily writing applications which process vast amounts of data (multi-terabyte data-sets) in-parallel on large clusters (thousands of nodes) of commodity hardware in a reliable, fault-tolerant manner.

<img src="https://i.stack.imgur.com/199Q1.png">

<h1><font color ='blue'>Practice in YARN and MapReduce</font></h1>

<h3>Part I: run the mapper and the reducer to obtain the word count of a input file</h3>

<b>- Transfering the program files to the edge:</b>

scp *.py e.luizfonseca-dsti@edge-1.au.adaltas.cloud:

<i>Info: here I was in the directory where the files I wanted to transfer were saved, if it's not the case give the path.</i>

The python files were given by Adaltas and are stored in Moodle.

<b>- Organizing the directories</b>

hdfs dfs -mkdir mr
hdfs dfs -mv raw/1342-0.txt raw/input.txt

<b>- Running the program</b>

yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \ 

    -file mapper.py -mapper "python mapper.py" \
    
    -file reducer.py -reducer "python reducer.py" \
    
    -input raw/input.txt \
    
     -output mr/output
     
<b>- Reading the result file</b> 

hdfs dfs -cat mr/output/part-00000

<h3>Part II: create a mapper and reducer script that will take the output of the wordcount as input, and output the most frequent word.</h3>

<b>- Developing the mapper and the reducer </b>

As we got as input the file with the counting the only task is to find the word with highest frequency, the algorithm is the same in the map and in the reduce stage, the difference is that during map we are distributing the task so we can have many files and in the reduce phase our job is to search amongst the files from the previous step the one that contains the most frequent word:

In [ ]:
import sys

current_word = None
highest_frequency_word = None
highest_frequency_count = 0
word = None

#input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    if count > highest_frequency_count:
        highest_frequency_count = count
        highest_frequency_word = word

#print on the file the first word with the highest frequency
print '%s\t%s' % (highest_frequency_word, highest_frequency_count)

<b>- Running the program</b>

yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \

-file mapper.py -mapper "python mapper_frequency.py" \

-file reducer.py -reducer "python mapper_frequency.py" \

-input raw/part-00000 \

 -output mr_freq/output
 
<b>- Reading the result file</b> 

hdfs dfs -cat mr_freq/output/part-00000